In [ ]:
import os
import sys

sys.path.append("../../common/src")
sys.path.append("../src")
os.chdir("../src")

In [ ]:
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent, AgentOutputParser
from langchain.agents.chat.base import ChatAgent
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI, VertexAI
from langchain.chat_models import ChatVertexAI, ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [ ]:
!export PROJECT_ID="gcp-mira-demo"
project = "gcp-mira-demo"
os.environ["PROJECT_ID"] = project
PROJECT_ID = project

In [ ]:
import config
from config import (VERTEX_LLM_TYPE_BISON_CHAT,
                    VERTEX_LLM_TYPE_BISON_TEXT,
                    OPENAI_LLM_TYPE_GPT3_5,
                    OPENAI_LLM_TYPE_GPT4,
                    COHERE_LLM_TYPE,
                    LLM_BACKEND_ROBOT_USERNAME,
                    LLM_BACKEND_ROBOT_PASSWORD)
from common.utils.token_handler import UserCredentials
config.TOOLS_SERVICE_BASE_URL = "https://gcp-mira-demo.cloudpssolutions.com/tools-service/api/v1"
config.auth_client = UserCredentials(LLM_BACKEND_ROBOT_USERNAME,
                              LLM_BACKEND_ROBOT_PASSWORD,
                              "https://gcp-mira-demo.cloudpssolutions.com")

In [ ]:
config.SERVICES['tools-service']['host']='https://gcp-mira-demo.cloudpssolutions.com'

In [ ]:
config.SERVICES['tools-service']['api_url_prefix']='https://gcp-mira-demo.cloudpssolutions.com/tools-service/api/v1'

In [ ]:
config.SERVICES['tools-service']

In [ ]:
from common.models import Agent
from common.models.agent import AgentType, UserPlan, PlanStep
from services.agents.agent_service import get_agent_config, get_plan_agent_config,agent_execute_plan

In [ ]:
from services.agents.agents import TaskAgent, StructuredTaskAgent, PlanAgent
#casey_plan_agent = TaskAgent(VERTEX_LLM_TYPE_BISON_TEXT)
#casey_plan_agent = TaskAgent(VERTEX_LLM_TYPE_BISON_CHAT)
#casey_plan_agent = TaskPlanAgent(OPENAI_LLM_TYPE_GPT3_5)
#casey_plan_agent = TaskAgent(OPENAI_LLM_TYPE_GPT4)
casey_plan_agent = StructuredTaskAgent(OPENAI_LLM_TYPE_GPT4)
#casey_plan_agent = TaskPlanAgent(COHERE_LLM_TYPE)

In [ ]:
prompt = """Compose and send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers"""

In [ ]:
user_id = "5nJrkPWa3D0yCKA853mD"

In [ ]:
#agent_executor.run(agent_inputs)

In [ ]:
from services.agents.agent_service import agent_plan, parse_plan

In [ ]:
plan = agent_plan("Plan", prompt, user_id)

user_plan = plan[1]
plan_steps=user_plan.plan_steps
chat_history = []

for step in plan_steps:
    description = PlanStep.find_by_id(step).description
    chat_history.append(description)

In [ ]:
tools = casey_plan_agent.get_tools()
print(tools)

agent = casey_plan_agent.load_agent()

prompt = """Execute the steps outlined in the chat_history input variable """
#prompt = """Compose and send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers"""
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=agent, 
    tools=tools,
    verbose=True)




In [ ]:
#chat_history[0]= '1. Use [query_tool] to [ find Medicaid applicants missing a pay stub ]'
#chat_history[2] = '3. Use [gmail_tool] to [send the composed email to the identified applicants]' 
#chat_history.insert(1,'Use [docs_tool] to [compose an email asking for a pay stub from their employers]')

In [ ]:
chat_history_string = ''.join(chat_history)
agent_inputs = {
    "input": prompt + 'chat_history:'+chat_history_string
}

In [ ]:
agent_inputs['input']

In [ ]:
result = agent_executor.run(agent_inputs)

In [ ]:
agent_execute_plan(agent_name='StructuredTask', prompt=agent_inputs['input'],user_plan=user_plan)